# Shopee - Price Match Guarantee
Determine if two products are the same by their images

* predicts which items are the same products.
* product matching

**Benifits:** 
The applications go far beyond Shopee or other retailers.
Your contributions to product matching could support:
* More accurate product categorization
* Uncover marketplace spam
* Aid shoppers to hunt for the very best deals.



In [ ]:
import imagehash
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
from PIL import Image
import plotly.express as px
import string
import re
import cv2
import warnings
warnings.filterwarnings("ignore")

#import geffnet
#from transformers import *
import os

#+++++++++++
import sys
sys.path = [
    '../input/geffnet-20200820'  
] + sys.path


# Loading and exploring data (EDA)

In [ ]:
# Images Directory

train_img_path = "../input/shopee-product-matching/train_images/"
test_img_path = "../input/shopee-product-matching/test_images/"


In [ ]:
train_data =pd.read_csv("../input/shopee-product-matching/train.csv")
test_data = pd.read_csv("../input/shopee-product-matching/test.csv")
sample = pd.read_csv('../input/shopee-product-matching/sample_submission.csv')
test_img = "../input/shopee-product-matching/test_images"

In [ ]:
train_data.head()

# Add Img full path as column: path

In [ ]:
train_data["path"] = [os.path.join(train_img_path,s) for s in train_data["image"]]
test_data["path"] = [os.path.join(test_img_path,s) for s in test_data["image"]]


In [ ]:
train_data.to_csv("train_data.csv",index=False)
test_data.to_csv("test_data.csv",index=False)

In [ ]:
train_data.head(3)

In [ ]:
nRow, nCol = train_data.shape
print('train_data:', {nRow},'rows ', {nCol},'columns')

In [ ]:
# find image_phash duplicates And visualize
dup_data = train_data["image_phash"].value_counts()
dup_data = pd.DataFrame(dup_data).reset_index(drop=False).rename(columns={"image_phash":"Occurences","index":"image_phash"})
dup_data["image_phash"] = dup_data["image_phash"].astype("str")
fig = px.bar(dup_data[:20],x="image_phash",y="Occurences",color= "Occurences",\
             title="duplicates image_phash ")
fig.update_layout(title ={"x":0.475,"y":0.9,"xanchor":"center","yanchor":"top"})
fig.show()

# **Disply top Occurences (duplicates) 20 records.**

In [ ]:
dup_data.head()

> image_phash fad28daa2ad05595 Occurences 26 let us check if all having the same pictuer

In [ ]:
#fad28daa2ad05595

images_filter = ['posting_id', 'image_phash', 'image', 'title', 'path']

images_filter2 = train_data[images_filter]

#df[ (df.column0 == thing1) and (df.column1 == thing2) ]

#filtered_data2 = filtered_data2.image_phash == "fad28daa2ad05595"

images_filter3 = images_filter2[images_filter2['image_phash'] == 'fad28daa2ad05595'] 
  
#print(df_new) 

# Filter data to image_phash = fad28daa2ad05595
# Title: sugarbaby

images_filter3.head()


In [ ]:
sugarbaby_item_len = len(images_filter3)
print(sugarbaby_item_len)

# Train Images

In [ ]:
plt.figure(figsize=(20,20))

for num,a in enumerate(images_filter3["path"]):
    plt.subplot(6,5,num+1)
    img = cv2.imread(a)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.axis("off")
    plt.title(images_filter3["posting_id"].iloc[num])
    plt.imshow(img)
    

# Observation on image_phash fad28daa2ad05595 
* 26 items have the same image and almost the same title.
* I will consider image_phash and the title with the image


In [ ]:
#04a00bccb7f1560e755f5f1def73db59.jpg
#1dbf735adb93d54e6340af02f9b5472e.jpg
#003c7129aef72ad3ae8af64eac5bfd29.jpg
#0f459b1e8aa89ee9e79b2ec95440e9c3.jpg

# Create the Hash Object of the first image
PictureHash1 = imagehash.average_hash(Image.open(train_img_path + '04a00bccb7f1560e755f5f1def73db59.jpg'))
print('Picture Hashing: ' + str(PictureHash1))

# Create the Hash Object of the second image
PictureHash2 = imagehash.average_hash(Image.open(train_img_path + '1dbf735adb93d54e6340af02f9b5472e.jpg'))
print('Picture Hashing: ' + str(PictureHash2))

if PictureHash1 == PictureHash2:
    print("Identical pictuers")
else:
    print("diffrent pictuers")



# I will start workting with test images
* i will check image_phash description  
* i will check the Average Hashing of the images

**Load the 3 Test Images**

In [ ]:
from PIL import *
from skimage import *
import imagehash

In [ ]:
plt.figure(figsize=(20,20))

for num,a in enumerate(test_data["path"]):
    
    plt.subplot(1,3,num+1)
    img = cv2.imread(a)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.imshow(img)
    plt.axis("off")

In [ ]:
test_data.head()

In [ ]:
# find image_phash duplicates And visualize
dup_data = test_data["image_phash"].value_counts()
dup_data = pd.DataFrame(dup_data).reset_index(drop=False).rename(columns={"image_phash":"Occurences","index":"image_phash"})
dup_data["image_phash"] = dup_data["image_phash"].astype("str")
fig = px.bar(dup_data[:25],x="image_phash",y="Occurences",color= "Occurences",\
             title="duplicates image_phash ")
fig.update_layout(title ={"x":0.475,"y":0.9,"xanchor":"center","yanchor":"top"})
fig.show()

# Submission CSV


In [ ]:
from collections import defaultdict

In [ ]:
every_phash = defaultdict(list)

In [ ]:
practicedf = train_data.iloc[:20,:]

In [ ]:
for num, row in enumerate(practicedf[['posting_id', 'image_phash']].values):
    every_phash[row[1]].append(row[0])

In [ ]:
phash_list = []

for num, row in enumerate(practicedf[['posting_id','image_phash']].values):
    pred = ""
    for a in every_phash[row[1]]:
        pred = pred + a + " "

    
    pred=pred[:-1] # delete last space
    
    phash_list.append(pred)

In [ ]:
phash_list[:20]

In [ ]:
practicedf["matches"] = phash_list
practicedf

In [ ]:
test_data

In [ ]:
every_phash = defaultdict(list)

In [ ]:
for num, row in enumerate(test_data[['posting_id', 'image_phash']].values):
    every_phash[row[1]].append(row[0])

In [ ]:
every_phash


In [ ]:
phash_list = []

for num, row in enumerate(test_data[['posting_id','image_phash']].values):
    pred = ""
    for a in every_phash[row[1]]:
        pred = pred + a + " "

    
    pred=pred[:-1] # delete last space
    
    phash_list.append(pred)

In [ ]:
test_data["matches"] = phash_list
test_data

In [ ]:
for num, row in enumerate(test_data[['posting_id', 'image_phash']].values):
    every_phash[row[1]].append(row[0])

In [ ]:
submission = test_data[["posting_id","matches"]]
submission

In [ ]:
submission.to_csv("submission.csv",index=False)

* **(Please consider upvoting if you find it useful to you)**
* **Switch off the internet from settings before submit**
* **Thanks**